In [18]:
!pip install openai
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.3
    Uninstalling openai-1.54.3:
      Successfully uninstalled openai-1.54.3


In [20]:
# Load and shuffle dataset
data = pd.read_excel("spanish.xlsx")  # Path to your dataset file
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate data by labels
depressed = data[data["Labels"] == 1]
not_depressed = data[data["Labels"] == 0]

# Balance the classes by taking an equal number of samples
min_samples = min(len(depressed), len(not_depressed))
depressed_sample = depressed.sample(min_samples, random_state=42)
not_depressed_sample = not_depressed.sample(min_samples, random_state=42)

# Combine the balanced samples
balanced_data = pd.concat([depressed_sample, not_depressed_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

# Extract sentences and labels
sentences = balanced_data["Tweets"]  # Text column
labels = balanced_data["Labels"]  # Label column

# Split data: 20% for testing
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2
)


In [43]:
import os
from openai import OpenAI
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import openai
from tqdm import tqdm  # Import tqdm for the progress bar

# Initialize OpenAI API client
api_key = "sk-proj-gwLjwM7RGbWgeLNKTQP9T3BlbkFJFtXCuOajKSgSq9Gp9qBt"  # Replace with your actual OpenAI API key
# Initialize OpenAI API key
openai.api_key = api_key  # Set the OpenAI API key
client = OpenAI(api_key=api_key)



# Function to get OpenAI's output for depression detection
def get_chatgpt_completion(prompt):
      response = client.chat.completions.create(
          messages=[
              {"role": "system", "content": "Classify the following text as 'depressed' or 'not depressed'. Respond with only 'depressed' or 'not depressed'."},
              {"role": "user", "content": prompt}
          ],
          model="gpt-4o",  # Using gpt-4o as specified
          max_tokens=10,
          temperature=0,
      )
      classification = response.choices[0].message.content.strip().lower()
      return classification



# Generate predictions with a progress bar
predictions = []
for sentence in tqdm(test_sentences, desc="Processing Predictions"):
    prediction = get_chatgpt_completion(sentence)
    predictions.append(1 if prediction == "depressed" else 0)
    time.sleep(5)  # Respect API rate limits

# Calculate and display metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

# Output results with 4 decimal places
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9000
Precision: 0.8333
Recall: 0.9091
F1 Score: 0.8695
